In [1]:
import matplotlib.pyplot as plt ## plotting package
import pandas as pd ## tabel package
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# Basis URL zonder offset
URL1 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_1961-1970'
URL2 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_1971-1980'
URL3 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_1981-1990'
URL4 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_1991-2000'
URL5 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_2001-2010'
URL6 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_2011-2020'
URL7 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_2021-present'
URLS = [URL1, URL2, URL3, URL4, URL5, URL6, URL7]

In [3]:
class Vlucht:
    id = 0           # index
    jaar = 0         # jaar van lancering
    massa = 0        # massa van capsule/payload
    vluchtnaam = ""  # naam van vlucht
    link = ""        # link naar vluchtpagina
    rocketlink = ""  # link naar launch vehicle pagina
    rocketmass = 0   # massa van launch vehicle
    rocketname = ""  # naam van launch vehicle

In [65]:
def lijst_halen(URL, vluchtlijst, start_id=0):
    id_nr = start_id
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    tabel = soup.find('tbody')
    rijen = tabel.find_all('tr')
    
    vluchten = vluchtlijst
    for rij in rijen[1:]:  #DIT AANPASSEN!!!
        if rij.find('th'): # check of de rij een header heeft
#             print('Header:' + rij.find('th').text)
            
            id_nr += 1
            vlucht = vlucht_halen(rij, id_nr)
            print(str(id_nr) + ". Naam: " + vlucht.vluchtnaam + ". Jaar: " + str(vlucht.jaar))
            
            if vlucht.rocketmass:
                massastring = str(vlucht.rocketmass)
            else:
                massastring = "Niet gevonden"

            print("Raket: " + vlucht.rocketname + ". Massa: " + massastring)
            
            vluchten.append(vlucht)
            print("*" * 40)
        else:
            continue

    
    return vluchten, id_nr

In [66]:
def vlucht_halen(rij, nr):
    vlucht = Vlucht()
    vlucht.id = nr
    
    cells = rij.find_all('td')
    
    # basisinformatie van lancering uit tabel halen
    datumnaam = cells[1].text.split()
    datum = " ".join(datumnaam[0:2])
    vlucht.datum = datum
    jaar  = datumnaam[2]
    vlucht.jaar = jaar
    naam  = " ".join(datumnaam[3:])
    vlucht.vluchtnaam = naam
    
    # link uit tabel halen, databoxes van betreffende pagina uitlezen
    urls = cells[1].find_all('a', href=True)
    link = 'https://en.wikipedia.org/' + urls[-1].get('href') # laatste link uit lijst halen
    vlucht.link = link
    
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    tabel = soup.find('table', {'class' : 'infobox'})
    headers = tabel.find_all('th', {'class' : 'infobox-label'})
    datarijen = tabel.find_all('td', {'class' : 'infobox-data'})
    
    #index ophalen van launch mass (lm)
    lm_index = 0
    for i in range(len(headers)):
        if headers[i].text == "Launch mass":
            lm_index = i
            break
            
    # launch mass inlezen
    try:
        massa_txt = datarijen[lm_index].text
        massa = massa_txt.split()[0]
        massa = massa.replace(',', '')
        massa = float(massa)
    except:
        massa = None
    vlucht.massa = massa
    
    # link ophalen van raket
    rocket_index = 0
    for i in range(len(headers)):
        if headers[i].text == "Rocket":
            rocket_index = i
            break
    
    if rocket_index > 0:
        urls = datarijen[rocket_index].find('a', href=True)
        rocketlink = 'https://en.wikipedia.org/' + urls.get('href')
        vlucht.rocketlink = rocketlink
        vlucht.rocketname = urls.text
        vlucht.rocketmass = raketmassa_ophalen(rocketlink)
    else:
        vlucht.rocketlink = None
    
    
    ## return
    return vlucht

In [67]:
unit_conversion = {'kg' : 1,
                   'kilogram' : 1,
                   'kilograms' : 1,
                   'lb' : 0.454,
                   'pounds' : 0.454,
                   'pound' : 0.454,
                   't' : 1000,
                   'tonne' : 1000
                   }
    

def raketmassa_ophalen(URL):
    mass = None   #default waarde, wordt veranderd als deze wordt gevonden
    
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    tabel = soup.find('table', {'class' : 'infobox hproduct'})
    headers = tabel.find_all('th', {'class' : 'infobox-label'})
    datarijen = tabel.find_all('td', {'class' : 'infobox-data'})
    rijen = tabel.find_all('tr')
    
    # range bepalen van de elementen die onder de size header vallen
    size_header_index = 0
    capacity_header_index = 0
    
    for i in range(len(rijen)):
        header = rijen[i].find('th')
        if header: # check of de de rij een header is
            if header.text == "Size":
                size_header_index = i
            elif header.text.strip() == "Capacity":
                capacity_header_index = i
    
    if size_header_index and capacity_header_index:   # check of de size header en cap header zijn gevonden
        # index van raket massa ophalen 
        for i in range(size_header_index, capacity_header_index):
            header = rijen[i].find('th')
            content = rijen[i].find('td')
            
            # check of de betreffende rij een header is
            if header:
                if header.text == "Mass": # check of het de juiste header is
                    mass_line = content.text.split()
                    unit_index = 0
                    unit = None
                    for key in unit_conversion.keys():
                        try:
                            unit_index = mass_line.index(key)
                            unit = key
                            break #breaks out of for loop if key is found
                        except:
                            continue

                    if unit_index != 0:
                        mass = mass_line[unit_index - 1]
                        mass = mass.replace(',', '')
                        mass = int(mass) * unit_conversion[unit]

    return mass

In [68]:
startmsg = "Starttijd: "
endmsg = "Eindtijd: "

def printtime(message):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(message + current_time)

In [69]:
vluchten = []    # initialiseer nieuwe vluchtlijst
id_huidig = 0    # initialiseer index

start = datetime.now()
printtime(startmsg)
print("X" * 40)

for u in URLS:
    vluchten, id_huidig = lijst_halen(u, vluchten, start_id=id_huidig)
    
    print("X" * 60)
    printtime(endmsg)
    minutes = (datetime.now() - start).seconds // 60    #floor division by 60
    seconds = (datetime.now() - start).seconds %  60    #modulo by 60 
    print("Tijd verstreken: " + str(minutes) + ' minuten, ' + str(seconds) + ' seconden.')
    print("X" * 60)

    


Starttijd: 16:32:27
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
1. Naam: Vostok 1. Jaar: 1961
Raket: Vostok-K. Massa: Niet gevonden
****************************************
2. Naam: Mercury-Redstone 3 (Freedom 7). Jaar: 1961
Raket: Redstone MRLV. Massa: 30000
****************************************
3. Naam: Mercury-Redstone 4 (Liberty Bell 7). Jaar: 1961
Raket: Redstone MRLV. Massa: 30000
****************************************
4. Naam: Vostok 2. Jaar: 1961
Raket: Vostok-K. Massa: Niet gevonden
****************************************
5. Naam: Mercury-Atlas 6 (Friendship 7). Jaar: 1962
Raket: Atlas LV-3B. Massa: 120000
****************************************
6. Naam: Mercury-Atlas 7 (Aurora 7). Jaar: 1962
Raket: Atlas LV-3B. Massa: 120000
****************************************
7. Naam: Vostok 3. Jaar: 1962
Raket: Vostok-K. Massa: Niet gevonden
****************************************
8. Naam: Vostok 4. Jaar: 1962
Raket: Vostok-K. Massa: Niet gevonden
*********************************

76. Naam: Soyuz T-2. Jaar: 1980
Raket: Soyuz-U. Massa: 313000
****************************************
77. Naam: Soyuz 37. Jaar: 1980
Raket: Soyuz-U. Massa: 313000
****************************************
78. Naam: Soyuz 38. Jaar: 1980
Raket: Soyuz-U. Massa: 313000
****************************************
79. Naam: Soyuz T-3. Jaar: 1980
Raket: Soyuz-U. Massa: 313000
****************************************
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Eindtijd: 16:33:21
Tijd verstreken: 0 minuten, 54 seconden.
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
80. Naam: Soyuz T-4. Jaar: 1981
Raket: Soyuz-U. Massa: 313000
****************************************
81. Naam: Soyuz 39. Jaar: 1981
Raket: Soyuz-U. Massa: 313000
****************************************
82. Naam: STS-1, Columbia. Jaar: 1981
Raket: . Massa: Niet gevonden
****************************************
83. Naam: Soyuz 40. Jaar: 1981
Raket: Soyuz-U. Massa: 313000
*********************************

148. Naam: STS-48, Discovery. Jaar: 1991
Raket: . Massa: Niet gevonden
****************************************
149. Naam: Soyuz TM-13. Jaar: 1991
Raket: Soyuz-U2. Massa: 297800
****************************************
150. Naam: STS-44, Atlantis. Jaar: 1991
Raket: . Massa: Niet gevonden
****************************************
151. Naam: STS-42, Discovery. Jaar: 1992
Raket: . Massa: Niet gevonden
****************************************
152. Naam: Soyuz TM-14. Jaar: 1992
Raket: Soyuz-U2. Massa: 297800
****************************************
153. Naam: STS-45, Atlantis. Jaar: 1992
Raket: . Massa: Niet gevonden
****************************************
154. Naam: STS-49, Endeavour. Jaar: 1992
Raket: . Massa: Niet gevonden
****************************************
155. Naam: STS-50, Columbia. Jaar: 1992
Raket: . Massa: Niet gevonden
****************************************
156. Naam: Soyuz TM-15. Jaar: 1992
Raket: Soyuz-U2. Massa: 297800
****************************************
157. Naam:

223. Naam: STS-92, Discovery. Jaar: 2000
Raket: . Massa: Niet gevonden
****************************************
224. Naam: Soyuz TM-31. Jaar: 2000
Raket: Soyuz-U. Massa: 313000
****************************************
225. Naam: STS-97, Endeavour. Jaar: 2000
Raket: . Massa: Niet gevonden
****************************************
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Eindtijd: 16:34:32
Tijd verstreken: 2 minuten, 5 seconden.
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
226. Naam: STS-98, Atlantis. Jaar: 2001
Raket: . Massa: Niet gevonden
****************************************
227. Naam: STS-102, Discovery. Jaar: 2001
Raket: . Massa: Niet gevonden
****************************************
228. Naam: STS-100, Endeavour. Jaar: 2001
Raket: . Massa: Niet gevonden
****************************************
229. Naam: Soyuz TM-32. Jaar: 2001
Raket: Soyuz-U. Massa: 313000
****************************************
230. Naam: STS-104, Atlantis. Jaar: 2001
Raket

294. Naam: Soyuz TMA-04M. Jaar: 2012
Raket: Soyuz-FG. Massa: 305000
****************************************
295. Naam: Shenzhou 9. Jaar: 2012
Raket: Long March 2F. Massa: 464000
****************************************
296. Naam: Soyuz TMA-05M. Jaar: 2012
Raket: Soyuz-FG. Massa: 305000
****************************************
297. Naam: Soyuz TMA-06M. Jaar: 2012
Raket: Soyuz-FG. Massa: 305000
****************************************
298. Naam: Soyuz TMA-07M. Jaar: 2012
Raket: Soyuz-FG. Massa: 305000
****************************************
299. Naam: Soyuz TMA-08M. Jaar: 2013
Raket: Soyuz-FG. Massa: 305000
****************************************
300. Naam: Soyuz TMA-09M. Jaar: 2013
Raket: Soyuz-FG. Massa: 305000
****************************************
301. Naam: Shenzhou 10. Jaar: 2013
Raket: Long March 2F. Massa: 464000
****************************************
302. Naam: Soyuz TMA-10M. Jaar: 2013
Raket: Soyuz-FG. Massa: 305000
****************************************
303. Naam: Soy

In [11]:
now1 = datetime.now()

# timediff = now.strftime("%H:%M:%S")
# now = datetime.now()
# current_time = now.strftime("%H:%M:%S")
# print(timediff)
# current_time = now.strftime("%H:%M:%S")
# print("Current Time =", current_time)

In [25]:
now2 = datetime.now()
diff = now2 - now1
print("hoi " + str(diff))
seconds = diff.seconds %60
minutes = diff.seconds //60
print('min: ' + str(minutes) + ' seconds: ' + str(seconds))

hoi 0:04:25.109618
min: 4 seconds: 25


In [51]:
a = 'lb)'
b = 'lb'
dic = [a, b, '15']
'lb' in b

True